## K-Means Clustering


**Overview**<br>
<a href="https://archive.ics.uci.edu/ml/datasets/online+retail">Online retail is a transnational data set</a> which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

The steps are broadly:
1. Read and understand the data
2. Clean the data
3. Prepare the data for modelling
4. Modelling
5. Final analysis and recomendation

# 1. Read and visualise the data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import datetime as dt

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

In [2]:
# read the dataset
retail_df = pd.read_csv("Online+Retail.csv", sep=",", encoding="ISO-8859-1", header=0)
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [3]:
# basics of the df
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [4]:
var_ad = pd.DataFrame(columns=['Variable','DType','Number of unique values','Number of nulls','Percentage of Nulls' , 'Values'])

In [5]:
def refresh_Var(df):
    var_ad = pd.DataFrame(columns=['Variable','DType','Number of unique values','Number of nulls','Percentage of Nulls' , 'Values'])
    for i, var in enumerate(df.columns):
        var_ad.loc[i] = [var,df[var].dtype, df[var].nunique(), df[var].isnull().sum(), round( (df[var].isnull().sum() / df.shape[0] ) * 100,2)  ,df[var].unique().tolist()]
    return var_ad

In [6]:
var_ad = refresh_Var(retail_df)
var_ad

,Variable,DType,Number of unique values,Number of nulls,Percentage of Nulls,Values
0,InvoiceNo,object,25900,0,0.00,"[536365, 536366, 536367, 536368, 536369, 53637..."
1,StockCode,object,4070,0,0.00,"[85123A, 71053, 84406B, 84029G, 84029E, 22752,..."
2,Description,object,4223,1454,0.27,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET..."
3,Quantity,int64,722,0,0.00,"[6, 8, 2, 32, 3, 4, 24, 12, 48, 18, 20, 36, 80..."
4,InvoiceDate,object,23260,0,0.00,"[01-12-2010 08:26, 01-12-2010 08:28, 01-12-201..."
5,UnitPrice,float64,1630,0,0.00,"[2.55, 3.39, 2.75, 7.65, 4.25, 1.85, 1.69, 2.1..."
6,CustomerID,float64,4372,135080,24.93,"[17850.0, 13047.0, 12583.0, 13748.0, 15100.0, ..."
7,Country,object,38,0,0.00,"[United Kingdom, France, Australia, Netherland..."


*Dropping Customer ID's in this case, because we can classify based on this only to identify customer*

**We are creating a new Numeric Field to get a new field based on which Clustering can be done** <br>
    - We cannot directly use Quantity, Price as it would not give a correct categorising.<br> 
    - Hence the need to derive Amount = Quantity * Price.

In [7]:
# new column: amount 
retail_df['amount'] = retail_df['Quantity']*retail_df['UnitPrice']
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34


# 2. Clean the data

In [8]:
# missing values
round(100*(retail_df.isnull().sum())/len(retail_df), 2)

InvoiceNo       0.00
StockCode       0.00
Description     0.27
Quantity        0.00
InvoiceDate     0.00
UnitPrice       0.00
CustomerID     24.93
Country         0.00
amount          0.00
dtype: float64

In [9]:
# drop all rows having missing values
retail_df = retail_df.dropna() 

retail_df.shape

(406829, 9)

# Prepare the data for modelling

- **R (Recency):** Number of days since last purchase
- **F (Frequency):** Number of tracsactions
- **M (Monetary):** Total amount of transactions (revenue contributed)

So we are trying to Build Something like <br> 
    1. CustomerID <br>
    2. Recency<br>
    3. Frequency<br>
    4. MonetaryValue<br> 

## Monetary

In [10]:
retail_df.groupby("CustomerID")["amount"].sum()

CustomerID
12346.0       0.00
12347.0    4310.00
12348.0    1797.24
12349.0    1757.55
12350.0     334.40
            ...   
18280.0     180.60
18281.0      80.82
18282.0     176.60
18283.0    2094.88
18287.0    1837.28
Name: amount, Length: 4372, dtype: float64

CustomerID is the Index Now, We need it as a separate Column

In [11]:
monetary_df = retail_df.groupby("CustomerID")["amount"].sum()
monetary_df = monetary_df.reset_index()
monetary_df.head()

,CustomerID,amount
0,12346.0,0.00
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40


## Freqeuncy

In [12]:
frequency_df = retail_df.groupby("CustomerID")["InvoiceNo"].count()
frequency_df = frequency_df.reset_index()
frequency_df.head()

,CustomerID,InvoiceNo
0,12346.0,2
1,12347.0,182
2,12348.0,31
3,12349.0,73
4,12350.0,17


Change the Column Name of InoviceNo to Frequency

In [13]:
frequency_df.columns = ["CustomerID","frequency"]
frequency_df.head()

,CustomerID,frequency
0,12346.0,2
1,12347.0,182
2,12348.0,31
3,12349.0,73
4,12350.0,17


## Recency

    1. To compute Number of days from the last transaction

In [14]:
print(retail_df["InvoiceDate"].max())
print(retail_df["InvoiceDate"].min())

31-10-2011 17:13
01-02-2011 08:23


    2.Get the maximum date and the Referance date. 
    3.We need to get the Date Difference of the Referance date the the transaction date for the customer. 
    4. Minimum Value of the difference for the Customer will give the recency of the customer.

### Convert Invoice Date to proper datetime

In [15]:
var_ad = refresh_Var(retail_df)
var_ad

,Variable,DType,Number of unique values,Number of nulls,Percentage of Nulls,Values
0,InvoiceNo,object,22190,0,0.0,"[536365, 536366, 536367, 536368, 536369, 53637..."
1,StockCode,object,3684,0,0.0,"[85123A, 71053, 84406B, 84029G, 84029E, 22752,..."
2,Description,object,3896,0,0.0,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET..."
3,Quantity,int64,436,0,0.0,"[6, 8, 2, 32, 3, 4, 24, 12, 48, 18, 20, 36, 80..."
4,InvoiceDate,object,20460,0,0.0,"[01-12-2010 08:26, 01-12-2010 08:28, 01-12-201..."
5,UnitPrice,float64,620,0,0.0,"[2.55, 3.39, 2.75, 7.65, 4.25, 1.85, 1.69, 2.1..."
6,CustomerID,float64,4372,0,0.0,"[17850.0, 13047.0, 12583.0, 13748.0, 15100.0, ..."
7,Country,object,37,0,0.0,"[United Kingdom, France, Australia, Netherland..."
8,amount,float64,4041,0,0.0,"[15.299999999999999, 20.34, 22.0, 15.3, 25.5, ..."


In [16]:
# convert to datetime
retail_df['InvoiceDate'] = pd.to_datetime(retail_df['InvoiceDate'], 
                                          format='%d-%m-%Y %H:%M')

In [17]:
var_ad = refresh_Var(retail_df)
var_ad

,Variable,DType,Number of unique values,Number of nulls,Percentage of Nulls,Values
0,InvoiceNo,object,22190,0,0.0,"[536365, 536366, 536367, 536368, 536369, 53637..."
1,StockCode,object,3684,0,0.0,"[85123A, 71053, 84406B, 84029G, 84029E, 22752,..."
2,Description,object,3896,0,0.0,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET..."
3,Quantity,int64,436,0,0.0,"[6, 8, 2, 32, 3, 4, 24, 12, 48, 18, 20, 36, 80..."
4,InvoiceDate,datetime64[ns],20460,0,0.0,"[1291191960000000000, 1291192080000000000, 129..."
5,UnitPrice,float64,620,0,0.0,"[2.55, 3.39, 2.75, 7.65, 4.25, 1.85, 1.69, 2.1..."
6,CustomerID,float64,4372,0,0.0,"[17850.0, 13047.0, 12583.0, 13748.0, 15100.0, ..."
7,Country,object,37,0,0.0,"[United Kingdom, France, Australia, Netherland..."
8,amount,float64,4041,0,0.0,"[15.299999999999999, 20.34, 22.0, 15.3, 25.5, ..."


### Compute the Max Date, Min Date and Set Referance Date

In [18]:
# compute the max date
max_date = max(retail_df['InvoiceDate'])
max_date

Timestamp('2011-12-09 12:50:00')

In [19]:
# compute the diff
retail_df['diff'] = max_date - retail_df['InvoiceDate']
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount,diff
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,373 days 04:24:00
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,373 days 04:24:00
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,373 days 04:24:00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,373 days 04:24:00
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,373 days 04:24:00


In [20]:
# recency
last_purchase = retail_df.groupby('CustomerID')['diff'].min()
last_purchase = last_purchase.reset_index()
last_purchase.head()

,CustomerID,diff
0,12346.0,325 days 02:33:00
1,12347.0,1 days 20:58:00
2,12348.0,74 days 23:37:00
3,12349.0,18 days 02:59:00
4,12350.0,309 days 20:49:00


## Merge